# Lab4-2. Elasticsearch

## Environment setup

In [ ]:
!pip install elasticsearch

In [ ]:
import os
os.chdir('/home/day4/24_SDS_SW_Architecture/day4')

In [ ]:
ELASTIC_USERNAME = "elastic"
ELASTIC_PASSWORD = "123456"
ELASTIC_PATH = "https://localhost:9200"

In [ ]:
# import libraries
from elasticsearch import Elasticsearch
from datetime import datetime
from pprint import pprint
from list_of_documents import list_of_nba_players

In [ ]:
# connect to Elasticsearch
def connect_to_elastic() -> Elasticsearch:
    client = Elasticsearch(
        ELASTIC_PATH,
        ca_certs = "/home/day4/24_SDS_SW_Architecture/day4/ca.crt",
        basic_auth=(ELASTIC_USERNAME, ELASTIC_PASSWORD),
    )
    return client
    
client = connect_to_elastic()
print(client.info())

## CRUD Operations

- Create
- Read
- Update
- Delete

1. Create: Dynamic Mapping

In [ ]:
# document to be indexed
doc = {
    "author": "Antoine de Saint-Exupéry",
    "text": "The Little Prince",
    "timestamp": datetime.now(),
}

# name of index
index_name = "test-index"

# if index does not exist, dynamically create it
if not client.indices.exists(index=index_name):
    client.indices.create(index=index_name) 

# insert document into index
# if id is not specified, it is automatically generated
response = client.index(index=index_name, body=doc)

pprint(response.body)

1. Create: Index Mapping

In [ ]:
INDEX_NAME = "nba_players"
mapping = {
    "first_name": {"type": "text"},
    "last_name": {"type": "text"},
    "date_of_birth": {"type": "date"},
    "position": {"type": "keyword"},
    "team": {"type": "keyword"},
    "avg_scoring": {"type": "float"},
    "avg_rebound": {"type": "float"},
    "avg_assist": {"type": "float"},
    "country": {"type": "keyword"},
}

# Create index
client.indices.create(index=INDEX_NAME, body={"mappings": {"properties": mapping}})

# Insert documents to the index
for id_doc, document in enumerate(list_of_nba_players):
    client.index(index=INDEX_NAME, id=id_doc, body=document)

2. Read

In [ ]:
# Read a document
response = client.get(index=INDEX_NAME, id=0)
pprint(response.body)

3. Update

In [ ]:
document_to_update = {
    "doc": {'first_name': 'Kyrie', 'last_name': 'Irving', 'date_of_birth': '1992-03-23', 
            'position': 'PG', 'team': 'Brooklyn Nets', 'avg_scoring': 24.9, 'avg_rebounds': 4.4, 
             'avg_assists': 6.4, 'country': 'Korea'}
}

print('# 기존')
pprint(client.get(index=INDEX_NAME, id=11).body)

# Update a document
client.update(index=INDEX_NAME, id=11, body=document_to_update)

print('\n# 변경')
pprint(client.get(index=INDEX_NAME, id=11).body)

4. Delete

In [ ]:
# Delete a document
result = client.delete(index=INDEX_NAME, id=11)

pprint(client.get(index=INDEX_NAME, id=11).body)

## Basic Query DSL (Domain Specific language)
- All query should be in json format

Match Query

In [ ]:
query = {
    "query": {
        "match": {
            "team": "Lakers"
        }
    }
}

# Search
response = client.search(index=INDEX_NAME, body=query)
pprint(response["hits"]["hits"])

Match_all Query

In [ ]:
query = {
    "query": {
        "match_all": {}
        # Default size: 10
    },
}

# Search
response = client.search(index=INDEX_NAME, body=query)
pprint(len(response["hits"]["hits"]))

In [ ]:
query = {
    "query": {
        "match_all": {},
    },
    "size": 20
}

# Search
response = client.search(index=INDEX_NAME, body=query)
pprint(len(response["hits"]["hits"]))

Boolean Query: "must"

In [ ]:
query = {
  "query": {
    "bool": {
      "must": [
        { "match": { "country": "USA" } },
        { "match": { "position": "PG" } }
      ]
    }
  }
}

# Search
response = client.search(index=INDEX_NAME, body=query)
for hit in response["hits"]["hits"]:
    print(hit["_source"]["first_name"], hit["_source"]["last_name"],
          hit["_source"]["country"], hit["_source"]["position"])

Range Query

In [ ]:
query = {
    "query": {
        "range": {
            "avg_scoring": {
                "gte": 20,
                "lte": 30
            }
        }
    }
}

# Search
response = client.search(index=INDEX_NAME, body=query)
for hit in response["hits"]["hits"]:
    print(hit["_source"]["first_name"], hit["_source"]["last_name"], hit["_source"]["avg_scoring"])

Sort Search Results

In [ ]:
query = {
    "sort": [
        {"avg_scoring": {"order": "desc"}}, {"avg_rebound": {"order": "desc"}}
    ],
    "size": 5
}

# Search
response = client.search(index=INDEX_NAME, body=query)
for hit in response["hits"]["hits"]:
    print(hit["_source"]["first_name"], hit["_source"]["last_name"], hit["_source"]["avg_scoring"], hit["_source"]["avg_rebound"])

## Exercise

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import json
from elasticsearch import Elasticsearch
from tqdm import tqdm

import os
os.chdir('/home/day4/24_SDS_SW_Architecture/day4')

In [ ]:
ELASTIC_USERNAME = "elastic"
ELASTIC_PASSWORD = "123456"
ELASTIC_PATH = "https://localhost:9200"

In [ ]:
# connect to Elasticsearch
def connect_to_elastic() -> Elasticsearch:
    client = Elasticsearch(
        ELASTIC_PATH,
        ca_certs = "/home/day4/24_SDS_SW_Architecture/day4/ca.crt",
        basic_auth=(ELASTIC_USERNAME, ELASTIC_PASSWORD),
    )
    return client
    
client = connect_to_elastic()
print(client.info())

In [ ]:
# Write mapping for the index
mapping = {
    "movieId": {
        "type": None
    },
    "title": {
        "type": None
    },
    "genres": {
        "type": None
    },
    "imdbId": {
        "type": None
    },
    "tmdbId": {
        "type": None
    },
    "userId": {
        "type": None
    },
    "rating": {
        "type": None
    },
    "timestamp": {
        "type": None
    }}

In [ ]:
# Create index
INDEX_NAME = "movie_review"

client = connect_to_elastic()
if not client.indices.exists(index=INDEX_NAME):
    client.indices.create(None)

In [ ]:
# Insert documents
with open('movie_data.json', 'r') as f:
    movie_json = json.load(f)
    for id_doc, document in enumerate(tqdm(movie_json)):
        client.index(None)

### Q1) What is the total number of reviews for films that contain the both "Action" and "Crime" genres?

In [ ]:
# Action and Crime
query = {
    "query": {
        None
    },
    "size": 1000
}

response = client.search(None)
print(len(response['hits']['hits']))

### Q2) What is the total number of reviews for films that contain the Sci-Fi genre and have a rating 4.0 or more but 5.0 or less and report the five movie in the list of results.

In [ ]:
query = {
    "query": {
        None
        }
    },
    "size": 1000
}

response = client.search(None)
print("Numbers of result:", response['hits']['total']['value'])

movie_list = []
for r in response['hits']['hits']:
    movie_list.append([r['_source']['title'], r['_source']['rating'], r['_source']['genres']])

print('\nResults')
for i in range(5):
    print(movie_list[i])

### Q3) Find the 10 lowest rated reviews between 2005 and 2010, in order.

In [ ]:
query = {
    "size": None,
    "query": {
        None
        }
    },
    "sort": [
        None
    ]
}

response = client.search(None)

from datetime import datetime
for hit in response['hits']['hits']:
    print(datetime.fromtimestamp(hit['_source']['timestamp']//1000), hit['_source']['title'], hit['_source']['rating'])

## 정답코드

In [ ]:
# Write mapping for the index
mapping = {
    "movieId": {
        "type": "integer"
    },
    "title": {
        "type": "keyword"
    },
    "genres": {
        "type": "text"
    },
    "imdbId": {
        "type": "integer"
    },
    "tmdbId": {
        "type": "integer"
    },
    "userId": {
        "type": "integer"
    },
    "rating": {
        "type": "float"
    },
    "timestamp": {
        "type": "date"
    }}

In [ ]:
# Create index
INDEX_NAME = "movie_review"

client = connect_to_elastic()
if not client.indices.exists(index=INDEX_NAME):
    client.indices.create(index=INDEX_NAME, body={'mappings': {'properties': mapping}})

In [ ]:
# Insert documents
with open('movie_data.json', 'r') as f:
    movie_json = json.load(f)
    for id_doc, document in enumerate(tqdm(movie_json)):
        client.index(index=INDEX_NAME, body=document, id=id_doc)

### Q1) What is the total number of reviews for films that contain the both "Action" and "Crime" genres?

In [ ]:
# Action and Crime
query = {
    "query": {
        "bool": {
            "must": [
                {"match": { "genres": "Action"}},
                {"match": { "genres": "Crime"}}
            ]
        }
    },
    "size": 1000
}

response = client.search(index=INDEX_NAME, body=query)
print(len(response['hits']['hits']))

### Q2) What is the total number of reviews for films that contain the Sci-Fi genre and have a rating 4.0 or more but 5.0 or less and report the five movie in the list of results.

In [ ]:
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "genres": "Sci-Fi"
                    }
                },
                {
                    "range": {
                        "rating": {
                            "gte": 4.0,
                            "lte": 5.0
                        }
                    }
                }
            ]
        }
    },
    "size": 1000
}

response = client.search(index=INDEX_NAME, body=query)
print("Numbers of result:", response['hits']['total']['value'])

movie_list = []
for r in response['hits']['hits']:
    movie_list.append([r['_source']['title'], r['_source']['rating'], r['_source']['genres']])

print('\nResults')
for i in range(5):
    print(movie_list[i])

### Q3) Find the 10 lowest rated reviews between 2005 and 2010, in order.

In [ ]:
query = {
    "size": 10,
    "query": {
        "range": {
            "timestamp": {
                "gte": "2005-01-01",
                "lte": "2010-12-31"
            }
        }
    },
    "sort": [
        {
            "rating": {
                "order": "asc"
            }
        }
    ]
}

response = client.search(index=INDEX_NAME, body=query)

from datetime import datetime
for hit in response['hits']['hits']:
    print(datetime.fromtimestamp(hit['_source']['timestamp']//1000), hit['_source']['title'], hit['_source']['rating'])